# Performance Comparison of MIP Solver Formulations for Capacitated VRP

This notebook compares the performance of different formulations for the Capacitated Vehicle Routing Problem (CVRP) using MIP solvers. CVRP is a variant of the Vehicle Routing Problem (VRP) where each customer has a demand and vehicles have capacity constraints. CVRP generalizes VRP, which can be considered as a special case of CVRP.

We implement each model using JijModeling, manage optimization results with MINTO, and solve the models using SCIP. Finally, we compare the performance of each model.

## MTZ formulation and Flow formulation

For CVRP, there are two major formulations: MTZ formulation (also called potential formulation) and Flow formulation.

### About Each Formulation

MTZ formulation and Flow formulation differ in how they track cumulative demand. MTZ formulation tracks cumulative demand at each customer node, while Flow formulation tracks flow amount on each edge.

- MTZ (Miller-Tucker-Zemlin) Formulation:
    MTZ formulation introduces continuous variables representing cumulative demand at each customer node to express subtour elimination constraints. This method has relatively fewer variables and a simpler model but weaker LP relaxation.

- Flow Formulation:
    Flow formulation introduces continuous variables representing flow amount on each edge and uses flow conservation laws to express subtour elimination constraints. This method has more variables but stronger LP relaxation than MTZ formulation.

Let's implement these two formulations and compare their performance.

## Common Parts of CVRP Formulation

Let's first implement the parts common to both models.

In [ ]:
import minto
import jijmodeling as jm
from minto.problems.cvrp import CVRPMTZ, CVRPFlow

## MTZ formulation

In [ ]:
CVRPMTZ().problem()

## Flow formulation

In [ ]:
CVRPFlow().problem()

## Random CVRP Instance Generation

We generate a random CVRP instance to test the models.

In [ ]:
n = 10
instance_data = CVRPFlow().random_data(n)
xy = instance_data["xy"]

## Performance Comparison

We compare the performance of the two formulations using the random CVRP instance.
The JijModeling implementation is converted to PySCIPOpt using `ommx_pyscipopt_adapter` and solved with SCIP. Results are logged using MINTO.

When logging using `.log_instance`, we assign names to each model to make later comparison easier.

For this tutorial we set `auto_saving=False`, but in practice it is recommended to use `auto_saving=True` for automatic log saving.

In [ ]:
import time
import ommx_pyscipopt_adapter as scip_ad

exp = minto.Experiment("cvrp", auto_saving=False, verbose_logging=False)

nlist = [8, 9, 10, 11, 12, 13]
for n in nlist:
    print(f"{n=}")
    with exp.run() as run:
        instance_data = CVRPFlow().random_data(n)
        run.log_parameter("n", n)
        intepreter = jm.Interpreter(instance_data)

        # MTZ model
        mtz_instance = intepreter.eval_problem(CVRPMTZ().problem())
        run.log_instance("mtz", mtz_instance)

        adapter = scip_ad.OMMXPySCIPOptAdapter(mtz_instance)
        scip_model = adapter.solver_input

        start = time.time()
        scip_model.optimize()
        scip_model.getStatus()
        elpsed_time = time.time() - start

        run.log_parameter("mtz_time", elpsed_time)

        mtz_solution = adapter.decode(scip_model)
        run.log_solution("mtz", mtz_solution)

        # Flow model
        intepreter = jm.Interpreter(instance_data)
        flow_instance = intepreter.eval_problem(CVRPFlow().problem())
        run.log_instance("flow", flow_instance)

        adapter = scip_ad.OMMXPySCIPOptAdapter(flow_instance)
        scip_model = adapter.solver_input

        start = time.time()
        scip_model.optimize()
        scip_model.getStatus()
        elpsed_time = time.time() - start

        run.log_parameter("flow_time", elpsed_time)

        flow_solution = adapter.decode(scip_model)
        run.log_solution("flow", flow_solution)

In [ ]:
exp.get_run_table()

In [ ]:
import matplotlib.pyplot as plt

param_table = exp.get_run_table()["parameter"]
plt.plot(param_table["n"], param_table["mtz_time"], "-o", label="MTZ")
plt.plot(param_table["n"], param_table["flow_time"], "-o", label="Flow")
plt.legend()
plt.xlabel("n")
plt.ylabel("time")
plt.show()

## Summary

In this notebook, we implemented MTZ and Flow formulations for the Capacitated Vehicle Routing Problem (CVRP) and compared their performance using a random CVRP instance. We found that the Flow formulation is faster than the MTZ formulation for the given instance.

It is interesting to note that despite having more variables, the Flow formulation performs better. This serves as a good example of how the effectiveness of branch-and-bound methods can vary depending on the constraint conditions and how easily solutions can be obtained.

However, the performance may vary depending on the instance characteristics. For example, different vehicle capacities could lead to significantly different results. Additionally, there are strengthened versions of the MTZ formulation available.

Feel free to experiment with different instance generation methods and strengthened versions of the MTZ formulation.